In [109]:
#예제 1 ------------6장---------------
from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')

In [110]:
#예제 1-2
jack = Customer('Park', 100)
jack

Customer(name='Park', fidelity=100)

In [111]:
#예제 1-3
class LineItem:
    """구매할 물품/갯수를 생성해서 총 가격을 반환"""
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price

    def total(self):
        return self.price * self.quantity

In [112]:
#예제 1-4
class Order:
    """고객(namedtuple) 및 LineItem 클래스의 인스턴스를 cart로 받아서 총 계산할 가격을 산출"""
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion
    def total(self):
        """__total 속성이 없으면 전체 계산할 값을 계산"""
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total
    def due(self):
        """할인 금액 차감"""
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion.discount(self)
        return self.total() - discount
    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())

In [113]:
#예제 1-5
class Promotion(ABC) :
    """할인 혜택 클래스들의 형태를 선언"""
    @abstractmethod
    def discount(self, order):
        """할인액을 구체적인 숫자로 반환"""
        pass

In [114]:
#예제 1-6
class FidelityPromo(Promotion):
    """충성도 점수가 1000점 이상인 고객에게 전체 5% 할인 적용"""

    def discount(self, order):
        return order.total() * .05 if order.customer.fidelity >= 1000 else 0
class BulkItemPromo(Promotion):
    """20개 이상의 동일 상품을 구입하면 10% 할인 적용"""

    def discount(self, order):
        discount = 0
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() * .1
        return discount
class LargeOrderPromo(Promotion):
    """10종류 이상의 물품을 구매하면 전체 7% 할인 적용"""

    def discount(self, order):
        distinct_items = {item.product for item in order.cart}
        if len(distinct_items) >= 10:
            return order.total() * .07
        return 0

In [115]:
#예제 2-1
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)
cart = [
    LineItem('banana', 4, .5),
    LineItem('apple', 10, 1.5),
    LineItem('watermellon', 5, 5.0)
]

In [116]:
#예제 2-2
Order(joe, cart, FidelityPromo())

<Order total: 42.00 due: 42.00>

In [117]:
#예제 2-3
Order(ann, cart, FidelityPromo())

<Order total: 42.00 due: 39.90>

In [118]:
#예제 2-4
banana_cart = [
    LineItem('banana', 30, .5),
    LineItem('apple', 10, 1.5)
]
Order(joe, banana_cart, BulkItemPromo())

<Order total: 30.00 due: 28.50>

In [119]:
#예제 2-5
long_order = [LineItem(str(item_code), 1, 1.0) for item_code in range(10)]
Order(joe, long_order, LargeOrderPromo())

<Order total: 10.00 due: 9.30>

In [120]:
#예제 2-6
Order(joe, cart, LargeOrderPromo())

<Order total: 42.00 due: 42.00>

In [121]:
#예제 3-1
class Order:
    """고객(nameedtuple) 및 LineItem 클래스의 인스턴스를 cart로 받아서 총 계산할 가격을 산출"""
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion
    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total
    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion(self)
        return self.total() - discount
    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())


In [122]:
#예제 3-2
def fidelity_promo(order):
    """충성도 점수가 1000점 이상인 고객에게 전체 5% 할인 적용"""
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0

def bulk_item_promo(order):
    """20개 이상의 동일 상품을 구입하면 10% 할인 적용"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * .1
    return discount

def large_order_promo(order):
    """10종류 이상의 물품을 구매하면 전체 7% 할인 적용"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * .07
    return 0

In [123]:
#예제 3-3
Order(joe, cart, fidelity_promo)

<Order total: 42.00 due: 42.00>

In [124]:
#예제 3-4
Order(ann, cart, fidelity_promo)

<Order total: 42.00 due: 39.90>

In [125]:
#예제 3-5
Order(joe, banana_cart, bulk_item_promo)

<Order total: 30.00 due: 28.50>

In [126]:
#예제 3-6
Order(joe, long_order, large_order_promo)

<Order total: 10.00 due: 9.30>

In [127]:
#예제 4
proms = [fidelity_promo, bulk_item_promo, large_order_promo]
def best_promo(order):
    """최대로 할인받은 금액을 반환"""
    return max(promo(order) for promo in proms)

In [128]:
#예제 5-1
Order(joe, long_order, best_promo)

<Order total: 10.00 due: 9.30>

In [129]:
#예제 5-2
Order(joe, banana_cart, best_promo)

<Order total: 30.00 due: 28.50>

In [130]:
#예제 5-3
Order(ann, cart, best_promo)

<Order total: 42.00 due: 39.90>

In [131]:
#예제 5-4
globals().keys()

dict_keys(['__name__', '__doc__', '__package__', '__loader__', '__spec__', '__builtin__', '__builtins__', '_ih', '_oh', '_dh', 'In', 'Out', 'get_ipython', 'exit', 'quit', '_', '__', '___', '_i', '_ii', '_iii', '_i1', '_i2', 'ABC', 'abstractmethod', 'namedtuple', 'Customer', '_i3', 'jack', '_3', '_i4', 'LineItem', '_i5', 'Order', '_i6', 'Promotion', '_i7', 'FidelityPromo', 'BulkItemPromo', 'LargeOrderPromo', '_i8', 'joe', 'ann', 'cart', '_i9', '_9', '_i10', '_10', '_i11', '_i12', 'fidelity_promo', 'bulk_item_promo', 'large_order_promo', '_i13', '_13', '_i14', '_14', '_i15', '_i16', 'banana_cart', '_16', '_i17', '_i18', '_i19', '_19', '_i20', '_i21', '_21', '_i22', '_i23', '_i24', '_i25', '_i26', '_i27', '_27', '_i28', '_28', '_i29', '_29', '_i30', '_i31', '_i32', '_32', '_i33', '_33', '_i34', '_34', '_i35', 'long_order', '_35', '_i36', '_i37', '_37', '_i38', '_i39', '_i40', '_i41', '_i42', '_i43', '_43', '_i44', '_44', '_i45', '_45', '_i46', '_46', '_i47', '_47', '_i48', '_i49', '_i50',

In [132]:
#예제 5-5
globals()['bulk_item_promo']

<function __main__.bulk_item_promo(order)>

In [133]:
#예제 6-1
promos = [globals()[name] for name in globals()
          if name.endswith('_promo')
          and name != 'best_promo']
promos

[<function __main__.fidelity_promo(order)>,
 <function __main__.bulk_item_promo(order)>,
 <function __main__.large_order_promo(order)>]

In [134]:
#예제 1-1 ------------7장---------------
def deco(func):
    def inner():
        print('running inner()')
    return inner

In [135]:
#예제 1-2
@deco
def target():
    print('running target()')

target() #실제로는 inner함수 실행

running inner()


In [136]:
#예제 1-3
target # inner을 가리킨다.

<function __main__.deco.<locals>.inner()>

In [137]:
#예제 2
registry = [] # 여기에 decorated function 추가
def register(func):
    print('running register(%s)' % func)
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')

@register
def f2():
    print('running f2()')

def f3():
    print('running f3()')

def main():
    print('running main()')
    print('registry ->', registry)
    f1()
    f2()
    f3()

if __name__ == '__main__':
    main()

running register(<function f1 at 0x7a204458aef0>)
running register(<function f2 at 0x7a204458ab00>)
running main()
registry -> [<function f1 at 0x7a204458aef0>, <function f2 at 0x7a204458ab00>]
running f1()
running f2()
running f3()


In [138]:
#예제 3-1
promos = []
def promotion(promo_func):
    """promotion() decorator는 promo_func를 promos 리스트에 추가한 후 그대로 반환"""
    promos.append(promo_func)
    return promo_func
@promotion
def fidelity(order):
    """충성도 점수가 1000점 이상인 고객에게 전체 5% 할인 적용"""
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0

@promotion
def bulk_item(order):
    """20개 이상의 동일 상품을 구입하면 10% 할인 적용"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * .1
    return discount

@promotion
def large_order(order):
    """10종류 이상의 물품을 구매하면 전체 7% 할인 적용"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * .07
    return 0

def best_promo(order):
    """최대로 할인받을 금액을 반환"""
    return max(promo(order) for promo in promos)

In [139]:
#예제 4
def f1(a):
    print(a)
    print(b)

f1(3)

3
9


In [140]:
#예제 5
b=6
def f2(a):
    print(a)
    print(b)
    b=9

f2(3)

3


UnboundLocalError: local variable 'b' referenced before assignment

In [141]:
#예제 6-1
b = 6
def f3(a):
    global b
    print(a)
    print(b)
    b = 9
f3(3)

3
6


In [142]:
#예제 6-2
b

9

In [143]:
#예제 6-3
f3(3)

3
9


In [144]:
#예제 6-4
b=30
f3(3)

3
30


In [145]:
#예제 7-1
class Averager():
    def __init__(self):
        self.series = []
    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total/len(self.series)

In [146]:
#예제 7-2
avg = Averager()
avg(10)

10.0

In [147]:
#예제 7-3
avg(11)

10.5

In [148]:
#예제 7-4
avg(12)

11.0

In [149]:
#예제 8-1
def make_averager():
    series = []
    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total/len(series)
    return averager


In [150]:
#예제 8-2
avg = make_averager()
avg(10)

10.0

In [151]:
#예제 8-3
avg(11)

10.5

In [152]:
#예제 8-4
avg(12)

11.0

In [153]:
#예제 9-1
avg.__code__.co_varnames

('new_value', 'total')

In [154]:
#예제 9-2
avg.__code__.co_freevars

('series',)

In [155]:
#예제 9-3
avg.__closure__

(<cell at 0x7a20448df760: list object at 0x7a204420c3c0>,)

In [156]:
#예제 9-4
avg.__closure__[0].cell_contents

[10, 11, 12]

In [157]:
#예제 10-1
def make_averager():
    count = 0
    total = 0
    def averager(new_value):
        count += 1
        total += new_value
        return total / count
    return averager

In [158]:
#예제 10-2
avg = make_averager()
avg(10)

UnboundLocalError: local variable 'count' referenced before assignment

In [159]:
#예제 11-1
def make_averager():
    count = 0
    total = 0
    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total / count
    return averager

In [160]:
#예제 11-2
avg = make_averager()
avg(10)

10.0

In [161]:
#예제 12
import time
def clock(func):
    def clocked(*args):
        t0 = time.perf_counter()
        result = func(*args)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked


In [162]:
#예제 13-1
@clock
def snooze(seconds):
    time.sleep(seconds)

@clock
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)

if __name__ == '__main__':
    print('*' * 40, 'Calling snooze(.123)')
    snooze(.123)
    print('*' * 40, 'Calling factorial(6)')
    print('6! =', factorial(6))

**************************************** Calling snooze(.123)
[0.12326117s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000086s] factorial(1) -> 1
[0.00004124s] factorial(2) -> 2
[0.00006938s] factorial(3) -> 6
[0.00009701s] factorial(4) -> 24
[0.00012428s] factorial(5) -> 120
[0.00015438s] factorial(6) -> 720
6! = 720


In [163]:
#예제 13-2
factorial.__name__

'clocked'

In [164]:
import time
import functools

def clock(func):
    @functools.wraps(func)  # 원래 함수의 이름과 문서 문자열 유지
    def clocked(*args, **kwargs):
        t0 = time.time()  # 시작 시간
        result = func(*args, **kwargs)  # 원래 함수 호출
        elapsed = time.time() - t0  # 경과 시간 계산

        arg_list = []
        name = func.__name__  # 함수 이름 가져오기
        if args:
            arg_list.append(', '.join(repr(arg) for arg in args))
        if kwargs:
            pairs = ['%s=%r' %(k,w) for k,w in sorted(kwargs.items())]
            arg_list.append(', '.join(pairs))
        arg_str = ', '.join(arg_list)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked

In [165]:
#예제 14-2
import time

@clock
def snooze(seconds):
    time.sleep(seconds)

@clock
def factorial(n):
    return 1 if n < 2 else n * factorial(n - 1)

if __name__ == '__main__':
    print('*' * 40, 'Calling snooze(.123)')
    snooze(.123)
    print('*' * 40, 'Calling factorial(6)')
    print('6! =', factorial(6))

**************************************** Calling snooze(.123)
[0.12320757s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000119s] factorial(1) -> 1
[0.00005889s] factorial(2) -> 2
[0.00009155s] factorial(3) -> 6
[0.00012159s] factorial(4) -> 24
[0.00015163s] factorial(5) -> 120
[0.00018096s] factorial(6) -> 720
6! = 720


In [166]:
#예제 14-3
factorial.__name__

'factorial'